## LongevityMap genes splitting

Source of original data: [https://genomics.senescence.info](https://genomics.senescence.info/longevity/)<br />
*(LongevityMap build 3, release date: 2017 June 24, number of genes: 884)*

In [1]:
import pandas as pd
import re
from pathlib import Path

In [2]:
debug_local = True#False
local = Path("..").resolve()
data  = local / "data"
input = data / "input"
output = data / "output"

In [3]:
# read original data into dataFrame
df = pd.read_csv(input / 'longevity_genes_original.csv')

print("Dimension:", df.shape)
df.head()

Dimension: (550, 7)


,id,Association,Population,Variant(s),Gene(s),PubMed,Unnamed: 6
0,1,non-significant,Dutch,HLA-B40,HLA-B,1859103,NaN
1,2,non-significant,Dutch,HLA-DRB5,HLA-DRB5,1859103,NaN
2,3,non-significant,Finnish,APOB,APOB,8018664,NaN
3,4,significant,Finnish,APOC3,APOC3,8018664,NaN
4,5,significant,Finnish,E2/E3/E4,APOE,8018664,NaN


<br />

### Clean some data

In [4]:
# if string in the column 'Variant(s)' contains comma at the end, delete this last character (ignore records that contains NaN)
print("In the explored column, number of records with comma at the end:",
      len(df[df['Variant(s)'].str.endswith(',', na=False)]), end='')

df['Variant(s)'] = df['Variant(s)'].map(lambda st: st[:-1] if st.endswith(',') else st, na_action='ignore')

# check that we eliminate all cases with comma at the end
assert len(df[df['Variant(s)'].str.endswith(',', na=False)]) == 0, "Something wrong with removing comma at the end of string"
print('  - cleaned')

In the explored column, number of records with comma at the end: 1  - cleaned


In [5]:
# if string in the column 'Variant(s)' contains two commas in a row, replace them by one comma
print("In the explored column, number of records containing two commas in a row:",
      len(df[df['Variant(s)'].str.contains(',,', na=False)]), end='')

df['Variant(s)'] = df['Variant(s)'].str.replace(',,', ',')

# check that we eliminate all cases with two commas in a row
assert len(df[df['Variant(s)'].str.contains(',,', na=False)]) == 0, "Something wrong with removing two commas in a row"
print('  - cleaned')

In the explored column, number of records containing two commas in a row: 2  - cleaned


<br />

### Preliminary separation of original dataFrame into several dataFrames

In [6]:
# Last columns of the dataFrame contains only NaN value.
# Make sure of this and delete this column as unnecessary.
assert df['Unnamed: 6'].isna().all(), "In the column 'Unnamed: 6' there is at least 1 record with value other than Nan"
del df['Unnamed: 6']

print("Dimension:", df.shape)
df.head()

Dimension: (550, 6)


,id,Association,Population,Variant(s),Gene(s),PubMed
0,1,non-significant,Dutch,HLA-B40,HLA-B,1859103
1,2,non-significant,Dutch,HLA-DRB5,HLA-DRB5,1859103
2,3,non-significant,Finnish,APOB,APOB,8018664
3,4,significant,Finnish,APOC3,APOC3,8018664
4,5,significant,Finnish,E2/E3/E4,APOE,8018664


In [7]:
# Seperate records that do not have values in the column 'Variant(s)'
df_var_nan = df[(df['Variant(s)'].isna())]

print("Dimension:", df_var_nan.shape)
df_var_nan.head()

Dimension: (16, 6)


,id,Association,Population,Variant(s),Gene(s),PubMed
255,258,significant,American (Caucasian),NaN,TP53,20824210
256,261,non-significant,American (Caucasian),NaN,TP53,20824210
275,280,significant,European,NaN,TP53,23286790
276,281,significant,European,NaN,TP53,23286790
277,282,significant,European,NaN,TP53,23286790


In [8]:
# rows that have some info in the column 'Variant(s)' - this is intermediate dataFrame for further splitting
df_var_not_nan = df[(df['Variant(s)'].notna())]

print("Dimension:", df_var_not_nan.shape)
df_var_not_nan.head()

Dimension: (534, 6)


,id,Association,Population,Variant(s),Gene(s),PubMed
0,1,non-significant,Dutch,HLA-B40,HLA-B,1859103
1,2,non-significant,Dutch,HLA-DRB5,HLA-DRB5,1859103
2,3,non-significant,Finnish,APOB,APOB,8018664
3,4,significant,Finnish,APOC3,APOC3,8018664
4,5,significant,Finnish,E2/E3/E4,APOE,8018664


In [9]:
# rows that have sting in the column 'Variant(s)'
# but this string does not contains characters 'rs' with two digits after them
df_var_not_rs = df_var_not_nan[~df_var_not_nan['Variant(s)'].str.contains('.*rs\d{2}.*', flags=re.IGNORECASE)]

print("Dimension:", df_var_not_rs.shape)
df_var_not_rs.head()

Dimension: (320, 6)


,id,Association,Population,Variant(s),Gene(s),PubMed
0,1,non-significant,Dutch,HLA-B40,HLA-B,1859103
1,2,non-significant,Dutch,HLA-DRB5,HLA-DRB5,1859103
2,3,non-significant,Finnish,APOB,APOB,8018664
3,4,significant,Finnish,APOC3,APOC3,8018664
4,5,significant,Finnish,E2/E3/E4,APOE,8018664


In [10]:
# rows that have sting in the column 'Variant(s)' that contains characters 'rs' with two digits after them
df_var_rs = df_var_not_nan[df_var_not_nan['Variant(s)'].str.contains('.*rs\d{2}.*', flags=re.IGNORECASE)]

print("Dimension:", df_var_rs.shape)
df_var_rs.tail()

Dimension: (214, 6)


,id,Association,Population,Variant(s),Gene(s),PubMed
539,G548,non-significant,Danish,"rs1685354,rs647126",UCP3,22743239
540,G549,non-significant,Jordanian,"rs2241766,rs266729",ADIPOQ,20201642
541,G550,non-significant,Italian,"rs6457931,rs1321312,rs4331968,rs9470367,rs6920...","PANDAR,CDKN1A,RAB44",20126416
543,G552,non-significant,Danish,"rs2866164,Q95H",MTTP,16015282
546,556,significant,American (Caucasian),rs1042714,ADRB2,20399803


In [11]:
# ensure that we did not lost something during splitting of the original dataFrame
assert len(df) == len(df_var_nan) + len(df_var_rs) + len(df_var_not_rs), "Something wrong with splitting"

<br />

### Splitting of records with several 'rs' into several records

In [12]:
# Number of variants in each record in dataFrame with rs-variants.
# It is calculated by the number of commas +1
nmb_repeats = (df_var_rs['Variant(s)'].str.count(',') + 1).tolist()

assert len(df_var_rs) == len(nmb_repeats), "Problem with calculation"
print('Length of list:', len(nmb_repeats))
nmb_repeats[-5:]

Length of list: 214


[2, 2, 31, 2, 1]

In [13]:
# dublicate records in the dataFrame 'df_var_rs' that have several variants in the column 'Variant(s)'
df_var_rs_extended = df_var_rs.loc[df_var_rs.index.repeat(nmb_repeats)]

print("Dimension:", df_var_rs_extended.shape)
df_var_rs_extended.tail()

Dimension: (3032, 6)


,id,Association,Population,Variant(s),Gene(s),PubMed
541,G550,non-significant,Italian,"rs6457931,rs1321312,rs4331968,rs9470367,rs6920...","PANDAR,CDKN1A,RAB44",20126416
541,G550,non-significant,Italian,"rs6457931,rs1321312,rs4331968,rs9470367,rs6920...","PANDAR,CDKN1A,RAB44",20126416
543,G552,non-significant,Danish,"rs2866164,Q95H",MTTP,16015282
543,G552,non-significant,Danish,"rs2866164,Q95H",MTTP,16015282
546,556,significant,American (Caucasian),rs1042714,ADRB2,20399803


In [14]:
# Create "list" of all variants in the column 'Variant(s)' in the dataFrame 'df_var_rs'.
# To speed up calculation, use generator.
gen_variants = (el for ls in df_var_rs['Variant(s)'] for el in ls.split(','))
gen_variants

<generator object <genexpr> at 0x7f9f74b51eb0>

In [15]:
# change content of the columns 'Variant(s)' by splitting data
df_var_rs_extended['Variant(s)'] = list(gen_variants)

print("Dimension:", df_var_rs_extended.shape)
df_var_rs_extended.tail()

Dimension: (3032, 6)


,id,Association,Population,Variant(s),Gene(s),PubMed
541,G550,non-significant,Italian,rs6457940,"PANDAR,CDKN1A,RAB44",20126416
541,G550,non-significant,Italian,rs2145047,"PANDAR,CDKN1A,RAB44",20126416
543,G552,non-significant,Danish,rs2866164,MTTP,16015282
543,G552,non-significant,Danish,Q95H,MTTP,16015282
546,556,significant,American (Caucasian),rs1042714,ADRB2,20399803


In [16]:
# in the column 'Variant(s)' of dataFrame 'df_var_rs_extended', if the first two characters are 'Rs' than change them to 'rs'
df_var_rs_extended['Variant(s)'] = df_var_rs_extended['Variant(s)'].map(lambda st: 'rs' + st[2:] if st.startswith('Rs') else st)

<br />

### Combining dataFrames and saving result to disk

In [17]:
# combine all dataFrames in one
df_result_all = pd.concat([df_var_rs_extended, df_var_not_rs, df_var_nan])

print("Dimension:", df_result_all.shape)
df_result_all.iloc[len(df_var_rs_extended)-10:len(df_var_rs_extended)+5]   # look at the middle of dataframe, where joint occured

Dimension: (3368, 6)


,id,Association,Population,Variant(s),Gene(s),PubMed
541,G550,non-significant,Italian,rs3176343,"PANDAR,CDKN1A,RAB44",20126416
541,G550,non-significant,Italian,rs3176344,"PANDAR,CDKN1A,RAB44",20126416
541,G550,non-significant,Italian,rs3176349,"PANDAR,CDKN1A,RAB44",20126416
541,G550,non-significant,Italian,rs876581,"PANDAR,CDKN1A,RAB44",20126416
541,G550,non-significant,Italian,rs6457938,"PANDAR,CDKN1A,RAB44",20126416
541,G550,non-significant,Italian,rs6457940,"PANDAR,CDKN1A,RAB44",20126416
541,G550,non-significant,Italian,rs2145047,"PANDAR,CDKN1A,RAB44",20126416
543,G552,non-significant,Danish,rs2866164,MTTP,16015282
543,G552,non-significant,Danish,Q95H,MTTP,16015282
546,556,significant,American (Caucasian),rs1042714,ADRB2,20399803


In [18]:
# save all results together
df_result_all.to_csv(output / 'longevity_genes_splitted_all.csv', index=False)

<br />

In [19]:
# save only records that contain 'rs' at the beginning
df_result_rs = df_var_rs_extended[df_var_rs_extended['Variant(s)'].str.startswith('rs')]

print("Dimension:", df_result_rs.shape)
df_result_rs.tail()

Dimension: (3025, 6)


,id,Association,Population,Variant(s),Gene(s),PubMed
541,G550,non-significant,Italian,rs6457938,"PANDAR,CDKN1A,RAB44",20126416
541,G550,non-significant,Italian,rs6457940,"PANDAR,CDKN1A,RAB44",20126416
541,G550,non-significant,Italian,rs2145047,"PANDAR,CDKN1A,RAB44",20126416
543,G552,non-significant,Danish,rs2866164,MTTP,16015282
546,556,significant,American (Caucasian),rs1042714,ADRB2,20399803


In [20]:
df_result_rs.to_csv(output / 'longevity_genes_splitted_rs.csv', index=False)

<br />

In [21]:
# save other records, that do not contain 'rs' at the beginning
df_result_not_rs = pd.concat([df_var_rs_extended[~df_var_rs_extended['Variant(s)'].str.startswith('rs')],
                              df_var_not_rs,
                              df_var_nan
                             ])

print("Dimension:", df_result_not_rs.shape)
df_result_not_rs

Dimension: (343, 6)


,id,Association,Population,Variant(s),Gene(s),PubMed
407,G414,non-significant,German,Q/H 95,MTTP,15911777
415,G423,non-significant,Ashkenazi Jewish,APM1+2019,ADIPOQ,18511746
415,G423,non-significant,Ashkenazi Jewish,APM+2019,ADIPOQ,18511746
508,G517,significant,Italian,-308G/A,TNF,18511747
516,G525,Non-significant,Danish,-373(A)n(T)m,IL6,15130757
...,...,...,...,...,...,...
369,375,significant,Finnish,NaN,TP53,12483296
373,379,significant,Italian,NaN,TP53,10463944
381,387,significant,"European (Danish, Finnish, South Italian and G...",NaN,TP53,24341918
547,557,non-significant,American (Caucasian),NaN,TP53,20824210


In [22]:
df_result_not_rs.to_csv(output /'longevity_genes_splitted_not_rs.csv', index=False)

<br />

In [23]:
# ensure that we did lost something
assert len(df_var_rs_extended) + len(df_var_not_rs) + len(df_var_nan) == len(df_result_all), "Something was lost"
assert len(df_result_all) == len(df_result_rs) + len(df_result_not_rs), "Something was lost (2)"
print('Done')

Done
